In [32]:
import pystan
import pandas as pd
import numpy as np

In [2]:
def calc_consecutive_failures(test_row, i):
    """This function calculates number of consecutive failures

    Parameters
    ----------
    test_row : array
            array of test runs with 0, 1, 12 values as not run, pass, fail respectively
    i : float, int
            index in array i

    Returns
    ----------
    result: int
            number of consecutive failures from index i
    """

    result = 0
    while i < len(test_row) and (test_row[i] == 12 or test_row[i] == 0):
        if test_row[i] == 12:
            result += 1
        i += 1
    return result


def calc_flakiness_score(test_row, ignore_failures_in_a_row=3):
    """This function calculates flakiness score as the number of edges divided by total runs.
    At google, If the test is failing three times in a row, then only it reported as real failures;
    otherwise, it's considered a flaky test.
    (https://testing.googleblog.com/2016/05/flaky-tests-at-google-and-how-we.html)
    Hence, we ignored three or more than three consecutive failures
    and test cases that are not run while calculating the flakiness score.

    We always consider label 13 as an edge.
    since currently, each failed test is retry, and if it's passed on a subsequent run it is considered as flaky.

    Parameters
    ----------
    test_row : array
            array of test runs with 0, 1, 12, 13 values as not run, pass, fail, flaky respectively

    Returns
    ----------
    flakiness: int
            Flakiness score lies between 0 and 100; 0 is no flakiness, and 100 is maximum flakiness.
    """
    flip_arr = []
    flips = 0
    i = 0
    ## ignore not run instances in the starting
    while i < len(test_row) and test_row[i] == 0:
        i += 1

    ## intializing last_passing variable value
    if i >= len(test_row):
        return 0, len(test_row)*[0]
    elif test_row[i] == 1:
        last_passing = True
    elif test_row[i] == 13:
        last_passing = True
        flips += 1
        flip_arr.append(1)
    elif test_row[i] == 12:
        last_passing = False
    else:
        last_passing = True
    considerd = 1
    i += 1

    while i < len(test_row):
        ## ignoring more than three consecutive failures
        ## If the test is consecutively failing for three or more than three runs,
        ## we considered did not consider it an edge.
        cf = calc_consecutive_failures(test_row, i)
        if cf >= ignore_failures_in_a_row:
            if i + cf >= len(test_row):
                flip_arr = flip_arr + [0]*(len(test_row)-i)
                break
            else:
                flip_arr = flip_arr + [0]*(cf)
                i = i+cf

        s = test_row[i]
        if s == 1:
            ## run is pass
            flip_arr.append(0)
            considerd += 1
            last_passing = True
        elif s == 0:
            ## not run
            flip_arr.append(0)
            pass
        elif s == 13:
            ## flaky
            flip_arr.append(1)
            flips += 1
            considerd += 1
            last_passing = True
        elif s == 12:
            ## run is fail
            considerd += 1
            if last_passing:
                flip_arr.append(1)
                flips += 1
            else:
                flip_arr.append(0)
            last_passing = False
        i += 1
    if considerd == 0:
        return 0, flip_arr
    ## multiplying by 2 since flakiness score lies between 0 and 100
    flakiness = (flips / (considerd)) * 100
    return 2*flakiness/100, flip_arr

In [47]:
orig_data = []
row_list = []
for i in range(1000):
    arr = np.random.choice([1, 12], size=(20,))
    orig_data.append(arr)
    ans = calc_flakiness_score(arr)
    row_list.append([ans[0]]+ans[1])
transform_df = pd.DataFrame(row_list)
orig_df = pd.DataFrame(orig_data)

In [48]:
orig_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,12,1,12,1,1,12,12,1,1,12,12,1,12,12,12,1,12,1,1
1,12,1,1,1,12,12,12,1,1,1,1,1,1,1,12,1,1,1,1,1
2,12,1,12,12,12,1,1,1,1,1,12,12,12,1,1,12,12,1,12,12
3,12,1,1,1,12,1,1,12,12,1,1,1,12,12,1,12,12,1,12,12
4,12,12,12,1,12,1,1,12,1,1,1,1,1,1,1,12,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,12,1,1,12,1,1,12,12,12,1,1,12,12,12,12,1,1,12,12
996,1,1,1,12,1,12,1,12,12,1,1,12,12,12,12,1,1,12,1,12
997,12,12,1,1,1,12,12,12,1,1,12,12,12,12,12,1,12,12,1,1
998,12,1,1,12,12,1,1,12,1,1,12,12,12,12,12,12,12,1,12,1


In [49]:
transform_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.588235,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0
1,0.625000,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1
2,0.400000,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0
3,0.461538,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0.625000,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6495,0.461538,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
6496,0.625000,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1
6497,0.166667,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6498,0.461538,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0


In [35]:
stan_beta = """  
data {
  int<lower=1> N;
  int<lower=1> K;
  int<lower=1> J;
  vector<lower=0,upper=1>[N] y;
  matrix[N,K] X;
  matrix[N,J] Z;
}

parameters {
  vector[K] beta;
  vector[J] gamma;
}

transformed parameters{
  vector<lower=0,upper=1>[N] mu;    
  vector<lower=0>[N] phi;           
  vector<lower=0>[N] A;             
  vector<lower=0>[N] B;

  for (i in 1:N) {
    mu[i]  = inv_logit(X[i,] * beta);   
    phi[i] = exp(Z[i,] * gamma);
  }

  A = mu .* phi;
  B = (1.0 - mu) .* phi;
}

model {
  // priors

  // likelihood
  y ~ beta(A, B);
}

generated quantities{
  vector[N] log_lik;
  vector[N] log_lik_rep;
  vector<lower=0,upper=1>[N] y_rep;
  real total_log_lik;
  real total_log_lik_rep;
  
  int<lower=0, upper=1> p_omni;

  for (n in 1:N) {
    log_lik[n] = beta_lpdf(y[n] | A[n], B[n]);
    y_rep[n] = beta_rng(A[n], B[n]);
    log_lik_rep[n] = beta_lpdf(y_rep[n] | A[n], B[n]);
  }

  total_log_lik = sum(log_lik);
  total_log_lik_rep = sum(log_lik_rep);

  p_omni = (total_log_lik_rep > total_log_lik);
}
"""

In [50]:
transform_df['beta'] = transform_df.iloc[:,1:].sum(axis=1)
transform_df['gamma'] = (transform_df.iloc[:,1:]==0).sum(axis=1)
X = transform_df[['beta', 'gamma']]
X[1] = 1
cols = X.columns.tolist()
cols = cols[-1:] + cols[:-1]
X = X[cols]
y = transform_df[0].tolist()

In [72]:
data = {                                                                                             
    'N': len(y),
    'K': 3,
    'J': 3,
    'y': y,
    'X': X,
    'Z': X
}                                                                                                         
sm = pystan.StanModel(model_code=stan_beta)                                                               
fit = sm.sampling(data=data, seed=1234)   

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_d08992c73844bb255a96b86f1fa6c4f9 NOW.
